In [5]:
import tensorflow as tf
#Vers. 1.0.0
print(tf.__version__)
import sys
#Should be above 3.5
#print (sys.version)     

import cv2
import numpy as np
import os
import glob
import pickle

import matplotlib.image as mpimg
from skimage import color

import random

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

from Helpers import get_files, load_image, separate_imgs, get_next_batch_from_disk_RGB, get_next_batch_from_disk_RGB_Nocrop, get_next_batch_from_disk_RGB_Gaussian

1.0.0


In [6]:
images_list = get_files("../Data/Extracted/*", '*.jpg')

In [7]:
len(images_list)

35950

In [8]:
images_list_test = get_files("../Data/Extracted/Test/", '*.jpg')

In [9]:
len(images_list_test)

49


# Building the Training set


In [10]:
# weight initialization
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1, seed=seed, name=name)
    return tf.Variable(initial)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape, name=name)
    return tf.Variable(initial)

def conv2d(scope, x, kernel_size, stride=1):
    W = weight_variable(kernel_size, "conv_W_" + scope)
    b = bias_variable(kernel_size[3:], "conv_b_" + scope)
    conv = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME')
    return  tf.nn.relu(conv +  b)

#From https://github.com/shekkizh/Colorization.tensorflow/blob/master/TensorflowUtils.py
def conv2d_transpose_strided(scope, x, kernel_size, stride=2, output_shape=None):
    W = weight_variable(kernel_size, "deconv_W_" + scope)
    b = bias_variable(kernel_size[3:], "deconv_b_" + scope)
    
    if output_shape is None:
        output_shape = x.get_shape().as_list()
        output_shape[1] *= 2
        output_shape[2] *= 2
        output_shape[3] = W.get_shape().as_list()[2]
        
    return tf.nn.conv2d_transpose(x, W, output_shape, strides=[1, stride, stride, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def batch_norm(scope, x, train=True, reuse=False):
    return tf.contrib.layers.batch_norm(x, center=True, scale=True, updates_collections=None, is_training=train, trainable=True, scope=scope)



In [11]:
def train_neural_network_colorization_nilboyarch(x):
    
    #conv1
    conv_num = 1

    temp_conv = conv2d('conv' + str(conv_num), x, [3, 3, 1, 64], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 64, 64], stride=2)
    conv_num += 1
    
    print(temp_conv.get_shape())

    #self.nilboy = temp_conv

    temp_conv = batch_norm('bn_1', temp_conv,train=is_training)
    
    print(temp_conv.get_shape())
    
    #conv2
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 64, 128], stride=1)
    conv_num += 1
    
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 128, 128], stride=2)
    conv_num += 1
    
    temp_conv = batch_norm('bn_2', temp_conv,train=is_training)
    
    
    print(temp_conv.get_shape())
    
    #conv3
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 128, 256], stride=1)
    conv_num += 1
    
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 256, 256], stride=1)
    conv_num += 1    

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 256, 256], stride=2)
    conv_num += 1

    temp_conv = batch_norm('bn_3', temp_conv, train=is_training)
    
    print(temp_conv.get_shape())
    
    
    #conv4
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 256, 512], stride=1)
    conv_num += 1
    
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = batch_norm('bn_4', temp_conv,train=is_training)
    
    print(temp_conv.get_shape())

    #conv5
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1


    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = batch_norm('bn_5', temp_conv,train=is_training)
    
    
    print(temp_conv.get_shape())
    
    #conv6
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1    

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1    

    temp_conv = batch_norm('bn_6', temp_conv,train=is_training) 
    
    
    print(temp_conv.get_shape())
    
    #conv7
    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 512, 512], stride=1)
    conv_num += 1

    temp_conv = batch_norm('bn_7', temp_conv,train=is_training)
    
    print(temp_conv.get_shape())
    
    
    #Dropout layer for OF
    temp_conv = tf.nn.dropout(temp_conv, keep_prob)
    
    print(temp_conv.get_shape())
    
    
    #conv8
    temp_conv = conv2d_transpose_strided('conv' + str(conv_num), temp_conv, [4, 4, 256, 512], stride=2)
    conv_num += 1    

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 256, 256], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 256, 256], stride=1)
    conv_num += 1
    
    temp_conv = batch_norm('bn_8', temp_conv,train=is_training)
    
    print(temp_conv.get_shape())

    
    #conv9
    temp_conv = conv2d_transpose_strided('conv' + str(conv_num), temp_conv, [3, 3, 128, 256], stride=2)
    conv_num += 1    

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 128, 128], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 128, 128], stride=1)
    conv_num += 1
    
    temp_conv = batch_norm('bn_9', temp_conv,train=is_training)
    
    print(temp_conv.get_shape())
    
    #conv10
    temp_conv = conv2d_transpose_strided('conv' + str(conv_num), temp_conv, [3, 3, 64, 128], stride=2)
    conv_num += 1    

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 64, 64], stride=1)
    conv_num += 1

    temp_conv = conv2d('conv' + str(conv_num), temp_conv, [3, 3, 64, 64], stride=1)
    conv_num += 1
    
    temp_conv = batch_norm('bn_10', temp_conv,train=is_training)
    
    print(temp_conv.get_shape())
    
    W_last = weight_variable([3, 3, 64, 3], 'conv_W_last')
    b_last = bias_variable([3], 'conv_b_last')
    last_cnn = tf.nn.sigmoid(tf.nn.conv2d(temp_conv, W_last, strides=[1, 1, 1, 1], padding='SAME') )
    print(last_cnn.get_shape())
    
    
    _lambda = 0.05;
    l1_regularizer = tf.contrib.layers.l1_regularizer(scale=_lambda, scope=None)
    regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
    
    
    #Loss computation
    gen_loss_mse = tf.reduce_mean(2 * tf.nn.l2_loss(last_cnn - y)) + regularization_penalty
    optimizer = tf.train.AdamOptimizer().minimize(gen_loss_mse)
    
    #pred image
    tf.add_to_collection('optimizer', optimizer)  
    tf.add_to_collection('loss', gen_loss_mse) 
    tf.add_to_collection('color_image', last_cnn)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    #Training
    for itr in range(hm_epochs):
        #image, color_images = get_next_batch(train_features, train_labels)
        image, color_images = get_next_batch_from_disk_RGB_Gaussian(images_list=images_list, batch_size=batch_size, crop_size=224)
        #print(image.shape)
        feed_dict = {x: image, y: color_images, keep_prob: dropout_keep_prob, is_training : True}
        

        sess.run(optimizer, feed_dict=feed_dict)
        
        if itr % 20 == 0:
            
            image_test, color_images_test = get_next_batch_from_disk_RGB_Gaussian(images_list=images_list_test, batch_size=batch_size, crop_size=224)
            feed_dict_test = {x: image_test, y: color_images_test, keep_prob: 1.0, is_training : False}
            
            mse = sess.run(gen_loss_mse, feed_dict=feed_dict_test)
            sess.run(optimizer, feed_dict=feed_dict)
            print("Step: %d, MSE: %g" % (itr, mse))
     
    saver.save(sess, model_path)
    
    sess.close()

In [12]:
#Global constants
seed = 56

#cycles of feed forward + backprop on all K-folded samples
hm_epochs = 40
dropout_keep_prob = 0.5

model_path = "./recent/model.ckpt"
save_dir = './recent/'

batch_size = 15

x = tf.placeholder(tf.float32, shape=[batch_size, 224, 224, 1], name = 'x')
y = tf.placeholder(tf.float32, shape=[batch_size, 224, 224, 3], name='y')
keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
is_training = tf.placeholder(tf.bool, name = 'is_training');
#dropout_prob = tf.placeholder('float', (), name = 'dropout_prob')


if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
    
train_neural_network_colorization_nilboyarch(x)

(15, 112, 112, 64)
(15, 112, 112, 64)
(15, 56, 56, 128)
(15, 28, 28, 256)
(15, 28, 28, 512)
(15, 28, 28, 512)
(15, 28, 28, 512)
(15, 28, 28, 512)
(15, 28, 28, 512)
(15, 56, 56, 256)
(15, 112, 112, 128)
(15, 224, 224, 64)
(15, 224, 224, 3)
Step: 0, MSE: 938652
Step: 20, MSE: 953446


In [ ]:
# weight initialization
def weight_variable(shape, name, dropout=1):
    initial = tf.truncated_normal(shape, stddev=0.1, seed=seed, name=name)
    return tf.Variable(initial)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape, name=name)
    return tf.Variable(initial)

def conv2d(x, W, strides=[1,1,1,1]):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

#From https://github.com/shekkizh/Colorization.tensorflow/blob/master/TensorflowUtils.py
def conv2d_transpose_strided(x, W, output_shape=None, stride=2):
    #print (x.get_shape())
    #print (W.get_shape())
    if output_shape is None:
        output_shape = x.get_shape().as_list()
        output_shape[1] *= 2
        output_shape[2] *= 2
        output_shape[3] = W.get_shape().as_list()[2]
    return tf.nn.conv2d_transpose(x, W, output_shape, strides=[1, stride, stride, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def batch_norm(scope, x, train=True, reuse=False):
    return tf.contrib.layers.batch_norm(x, center=True, scale=True, updates_collections=None, is_training=train, trainable=True, scope=scope)



In [5]:
def train_neural_network_colorization_sig(x):
    
    strides1 = [1, 1, 1, 1]
    strides2 = [1, 2, 2, 1]
    
    # first convolutional layer
    W_conv1 = weight_variable([3, 3, 1, 64], "W_conv1")
    b_conv1 = bias_variable([64], "b_conv1")

    h_conv1 = tf.nn.relu6(conv2d(x, W_conv1, strides2) + b_conv1)
    
    print(h_conv1.get_shape())
    
    #Second CNN layer
    W_conv2 = weight_variable([3, 3, 64, 128], "W_conv2")
    b_conv2 = bias_variable([128], "b_conv2")

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    
    print(h_conv2.get_shape())
    
    #Third CNN Layer
    W_conv3 = weight_variable([3, 3, 128, 128], "W_conv3")
    b_conv3 = bias_variable([128], "b_conv3")

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, strides2) + b_conv3)
    
    print(h_conv3.get_shape())
    
    #Fourth CNN Layer
    W_conv4 = weight_variable([3, 3, 128, 256], "W_conv4")
    b_conv4 = bias_variable([256], "b_conv4")

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4 ) + b_conv4)
    
    print(h_conv4.get_shape())
    
    #Fifth CNN Layer
    W_conv5 = weight_variable([3, 3, 256, 256], "W_conv5")
    b_conv5 = bias_variable([256], "b_conv5")

    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, strides2) + b_conv5)
    
    print(h_conv5.get_shape())
    
    #Sixth CNN Layer
    W_conv6 = weight_variable([3, 3, 256, 512], "W_conv6")
    b_conv6 = bias_variable([512], "b_conv6")

    h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)
    
    print(h_conv6.get_shape())
    
    #Mid Level features
    W_midlvl1 = weight_variable([3, 3, 512, 512], "W_midlvl1")
    b_midlvl1 = bias_variable([512], "b_midlvl1")

    h_midlvl1 = tf.nn.relu(conv2d(h_conv6, W_midlvl1) + b_midlvl1)
    #h_pool_midlvl1 = max_pool_2x2(h_midlvl1)

    print(h_midlvl1.get_shape())
    
    W_midlvl2 = weight_variable([3, 3, 512, 256], "W_midlvl2")
    b_midlvl2 = bias_variable([256], "b_midlvl2")

    h_midlvl2 = tf.nn.relu(conv2d(h_midlvl1, W_midlvl2) + b_midlvl2)

    print(h_midlvl2.get_shape())
    
    #Dropout layer
    h_drop = tf.nn.dropout(h_midlvl2, keep_prob)

    #Colorization Network
    
    #Transposed CNN Layer 1
    W_transpose_conv1 = weight_variable([3, 3, 128, 256], "W_transpose_conv1")
    b_transpose_conv1 = bias_variable([128], "b_transpose_conv1")

    h_conv_t_4 = conv2d_transpose_strided(h_drop, W_transpose_conv1) + b_transpose_conv1
    
    print(h_conv_t_4.get_shape())
    
    # CNN conv
    
    W_colorizer_CNN1 = weight_variable([3, 3, 128 , 128 ], "W_colorizer_CNN1")
    b_colorizer_CNN1 = bias_variable([128 ], "b_colorizer_CNN1")

    h_colorizer_conv1 = tf.nn.relu6(conv2d(h_conv_t_4, W_colorizer_CNN1) + b_colorizer_CNN1)
    
    print(h_colorizer_conv1.get_shape())

    #Transposed CNN Layer 2
    W_transpose_conv2 = weight_variable([3, 3, 64, 128], "W_transpose_conv2")
    b_transpose_conv2 = bias_variable([64], "b_transpose_conv2")

    h_conv_t_5 = conv2d_transpose_strided(h_colorizer_conv1, W_transpose_conv2) + b_transpose_conv2
    
    print(h_conv_t_5.get_shape())
      
    #Transposed CNN Layer 3
    W_transpose_conv3 = weight_variable([3, 3, 3, 64], "W_transpose_conv3")
    b_transpose_conv3 = bias_variable([3], "b_transpose_conv3")

    h_conv_t_6 = tf.add(conv2d_transpose_strided(h_conv_t_5, W_transpose_conv3), b_transpose_conv3)
    
    print(h_conv_t_6.get_shape())
    
    W_last = weight_variable([3, 3, 3, 3], "W_last")
    b_last = bias_variable([3], "b_last")
    last_cnn = tf.nn.sigmoid(conv2d(h_conv_t_6, W_last) + b_last, name="color_image")
    
    print(last_cnn.get_shape())
    
    
    #to_save = [W_conv1, b_conv1, W_conv2, b_conv2, W_conv3, b_conv3, W_transpose_conv1, b_transpose_conv1, W_transpose_conv2, b_transpose_conv2, W_transpose_conv3, b_transpose_conv3]    
    
    '''_lambda = 0;
  
    
    l1_regularizer = tf.contrib.layers.l1_regularizer(scale=_lambda, scope=None)
   
    weights = [W_conv1, W_conv2, W_conv3, W_conv4, W_conv5, W_conv6, W_midlvl1, W_midlvl2,
                    W_transpose_conv1, W_colorizer_CNN1, W_transpose_conv2, W_transpose_conv3, W_last]
    
    regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, weights)
    '''
    
    #Loss computation
    gen_loss_mse = tf.reduce_mean(2 * tf.nn.l2_loss(last_cnn - y)) #+ regularization_penalty
    optimizer = tf.train.AdamOptimizer().minimize(gen_loss_mse)
    
    #pred image
    tf.add_to_collection('optimizer', optimizer)  
    tf.add_to_collection('loss', gen_loss_mse) 
    tf.add_to_collection('color_image', last_cnn)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    #Training
    for itr in range(hm_epochs):
        #image, color_images = get_next_batch(train_features, train_labels)
        image, color_images = get_next_batch_from_disk_RGB(images_list=images_list, batch_size=batch_size)
        #print(image.shape)
        feed_dict = {x: image, y: color_images, keep_prob: dropout_keep_prob}
        sess.run(optimizer, feed_dict=feed_dict)
        
        if itr % 100 == 0:
            mse = sess.run(gen_loss_mse, feed_dict=feed_dict)
            sess.run(optimizer, feed_dict=feed_dict)
            print("Step: %d, MSE: %g" % (itr, mse))
     
    saver.save(sess, model_path)
    
    sess.close()

In [6]:
#Global constants
seed = 56

#cycles of feed forward + backprop on all K-folded samples
hm_epochs = 1000
dropout_keep_prob = 0.5

model_path = "./test3/model.ckpt"
save_dir = './test3/'

batch_size = 80

x = tf.placeholder(tf.float32, shape=[batch_size, 64, 64, 1], name = 'x')
y = tf.placeholder(tf.float32, shape=[batch_size, 64, 64, 3], name='y')
keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
#dropout_prob = tf.placeholder('float', (), name = 'dropout_prob')


if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
    
train_neural_network_colorization_sig(x)

(80, 32, 32, 64)
(80, 32, 32, 128)
(80, 16, 16, 128)
(80, 16, 16, 256)
(80, 8, 8, 256)
(80, 8, 8, 512)
(80, 8, 8, 512)
(80, 8, 8, 256)
(80, 16, 16, 128)
(80, 16, 16, 128)
(80, 32, 32, 64)
(80, 64, 64, 3)
(80, 64, 64, 3)
Step: 0, MSE: 163609
Step: 100, MSE: 58441.2
Step: 200, MSE: 54213.9
Step: 300, MSE: 53787.1
Step: 400, MSE: 56334.3
Step: 500, MSE: 51149.2
Step: 600, MSE: 56110.1
Step: 700, MSE: 49822.7
Step: 800, MSE: 52687.2
Step: 900, MSE: 51984.4


In [11]:
image, color_images = get_next_batch_from_disk_RGB(images_list=images_list, batch_size=batch_size)
color_images.shape

(1, 224, 182, 3)

In [ ]:
'''

#tf.contrib.layers.apply_regularization(tf.contrib.layers.l1_regularizer(_lambda), weights_list=
#                                     [W_conv1, W_conv2, ])
#vars1  = tf.trainable_variables() 
#lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in vars1
            #if 'bias' not in v.name ]) * _lambda


#weights = [v for v in tf.trainable_variables() if 'bias' not in v.name]

'''

In [6]:
def train_neural_network_colorization_sig2(x):
    
    strides1 = [1, 1, 1, 1]
    strides2 = [1, 2, 2, 1]
    
    # first convolutional layer
    W_conv1 = weight_variable([3, 3, 1, 64], "W_conv1")
    b_conv1 = bias_variable([64], "b_conv1")

    h_conv1 = tf.nn.relu6(conv2d(x, W_conv1, strides2) + b_conv1)
    
    print(h_conv1.get_shape())
    
    #Second CNN layer
    W_conv2 = weight_variable([3, 3, 64, 128], "W_conv2")
    b_conv2 = bias_variable([128], "b_conv2")

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    
    print(h_conv2.get_shape())
    
    #Third CNN Layer
    W_conv3 = weight_variable([3, 3, 128, 128], "W_conv3")
    b_conv3 = bias_variable([128], "b_conv3")

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, strides2) + b_conv3)
    
    print(h_conv3.get_shape())
    
    #Fourth CNN Layer
    W_conv4 = weight_variable([3, 3, 128, 256], "W_conv4")
    b_conv4 = bias_variable([256], "b_conv4")

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4 ) + b_conv4)
    
    print(h_conv4.get_shape())
    
    #Fifth CNN Layer
    W_conv5 = weight_variable([3, 3, 256, 256], "W_conv5")
    b_conv5 = bias_variable([256], "b_conv5")

    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, strides2) + b_conv5)
    
    print(h_conv5.get_shape())
    
    #Sixth CNN Layer
    W_conv6 = weight_variable([3, 3, 256, 512], "W_conv6")
    b_conv6 = bias_variable([512], "b_conv6")

    h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)
    
    print(h_conv6.get_shape())
    
    #Mid Level features
    W_midlvl1 = weight_variable([3, 3, 512, 512], "W_midlvl1")
    b_midlvl1 = bias_variable([512], "b_midlvl1")

    h_midlvl1 = tf.nn.relu(conv2d(h_conv6, W_midlvl1) + b_midlvl1)
    #h_pool_midlvl1 = max_pool_2x2(h_midlvl1)

    print(h_midlvl1.get_shape())
    
    W_midlvl2 = weight_variable([3, 3, 512, 256], "W_midlvl2")
    b_midlvl2 = bias_variable([256], "b_midlvl2")

    h_midlvl2 = tf.nn.relu(conv2d(h_midlvl1, W_midlvl2) + b_midlvl2)

    print(h_midlvl2.get_shape())

    #Colorization Network
    
    #Upsample
    h_conv_t_4 = tf.image.resize_images(h_midlvl2, [64, 64], method=1) #1=nearest neighbour
    print("rs 1")
    print(h_conv_t_4.get_shape())
    
    # CNN conv
    
    W_colorizer_CNN1 = weight_variable([3, 3, 256 , 128], "W_colorizer_CNN1")
    b_colorizer_CNN1 = bias_variable([128], "b_colorizer_CNN1")

    h_colorizer_conv1 = tf.nn.relu6(conv2d(h_conv_t_4, W_colorizer_CNN1) + b_colorizer_CNN1)
    
    print(h_colorizer_conv1.get_shape())
    
    
    #Upsample
    h_conv_t_5 = tf.image.resize_images(h_colorizer_conv1, [128, 128], method=1) #1=nearest neighbour
    print("rs 2")
    print(h_conv_t_5.get_shape())
    
    
    # CNN conv
    W_colorizer_CNN2 = weight_variable([3, 3, 128 , 64], "W_colorizer_CNN2")
    b_colorizer_CNN2 = bias_variable([64], "b_colorizer_CNN2")

    h_colorizer_conv2 = tf.nn.relu6(conv2d(h_conv_t_5, W_colorizer_CNN2) + b_colorizer_CNN2)
    
    print(h_colorizer_conv2.get_shape())
    
    
    #Upsample
    h_conv_t_6 = tf.image.resize_images(h_colorizer_conv2, [256, 256], method=1) #1=nearest neighbour
    print("rs 3")
    print(h_conv_t_6.get_shape())
    
    # CNN conv
    W_colorizer_CNN3 = weight_variable([3, 3, 64 , 3], "W_colorizer_CNN3")
    b_colorizer_CNN3 = bias_variable([3], "b_colorizer_CNN3")

    h_colorizer_conv3 = tf.nn.relu6(conv2d(h_conv_t_6, W_colorizer_CNN3) + b_colorizer_CNN3)
    print(h_colorizer_conv3.get_shape())
    
    
    W_last = weight_variable([3, 3, 3, 3], "W_last")
    b_last = bias_variable([3], "b_last")
    last_cnn = tf.nn.sigmoid(conv2d(h_colorizer_conv3, W_last) + b_last, name="color_image")
    
    
    print(last_cnn.get_shape())

    #to_save = [W_conv1, b_conv1, W_conv2, b_conv2, W_conv3, b_conv3, W_transpose_conv1, b_transpose_conv1, W_transpose_conv2, b_transpose_conv2, W_transpose_conv3, b_transpose_conv3]    
    
    _lambda = 0.005;
  
    
    l1_regularizer = tf.contrib.layers.l1_regularizer(scale=_lambda, scope=None)
   
    weights = [W_conv1, W_conv2, W_conv3, W_conv4, W_conv5, W_conv6, W_midlvl1, W_midlvl2,
                    W_colorizer_CNN1, W_colorizer_CNN2,
                        W_colorizer_CNN3, W_last]
    
    regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, weights)
    
    #Loss computation
    gen_loss_mse = tf.reduce_mean(2 * tf.nn.l2_loss(last_cnn - y)) + regularization_penalty
    optimizer = tf.train.AdamOptimizer().minimize(gen_loss_mse)
    
    #pred image
    tf.add_to_collection('optimizer', optimizer)  
    tf.add_to_collection('loss', gen_loss_mse) 
    tf.add_to_collection('color_image', last_cnn)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    #Training
    for itr in range(hm_epochs):
        #image, color_images = get_next_batch(train_features, train_labels)
        image, color_images = get_next_batch_from_disk_RGB_Nocrop(images_list=images_list, batch_size=batch_size)
        #print(image.shape)
        feed_dict = {x: image, y: color_images}
        sess.run(optimizer, feed_dict=feed_dict)
        
        if itr % 100 == 0:
            mse = sess.run(gen_loss_mse, feed_dict=feed_dict)
            sess.run(optimizer, feed_dict=feed_dict)
            print("Step: %d, MSE: %g" % (itr, mse))
     
    saver.save(sess, model_path)
    
    sess.close()

In [7]:
#Global constants
seed = 56

#cycles of feed forward + backprop on all K-folded samples
hm_epochs = 1000

model_path = "./test5/model.ckpt"
save_dir = './test5/'

batch_size = 1

x = tf.placeholder(tf.float32, shape=[batch_size, 256, 256, 1], name = 'x')
y = tf.placeholder(tf.float32, shape=[batch_size, 256, 256, 3], name='y')
#dropout_prob = tf.placeholder('float', (), name = 'dropout_prob')

if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
    
train_neural_network_colorization_sig2(x)

(1, 128, 128, 64)
(1, 128, 128, 128)
(1, 64, 64, 128)
(1, 64, 64, 256)
(1, 32, 32, 256)
(1, 32, 32, 512)
(1, 32, 32, 512)
(1, 32, 32, 256)
rs 1
(1, 64, 64, 256)
(1, 64, 64, 128)
rs 2
(1, 128, 128, 128)
(1, 128, 128, 64)
rs 3
(1, 256, 256, 64)
(1, 256, 256, 3)
(1, 256, 256, 3)
Step: 0, MSE: 8138.69
Step: 100, MSE: 6891.7
Step: 200, MSE: 6383.14
Step: 300, MSE: 6104.76
Step: 400, MSE: 5930.78
Step: 500, MSE: 5818.62
Step: 600, MSE: 5740.91
Step: 700, MSE: 5680.94
Step: 800, MSE: 5633.58
Step: 900, MSE: 5586.99


In [ ]:
'''
def train_neural_network_colorization_sig(x):
    
    strides1 = [1, 1, 1, 1]
    strides2 = [1, 2, 2, 1]
    
    # first convolutional layer
    W_conv1 = weight_variable([3, 3, 1, 64], "W_conv1")
    b_conv1 = bias_variable([64], "b_conv1")

    h_conv1 = tf.nn.relu6(conv2d(x, W_conv1, strides2) + b_conv1)
    
    print(h_conv1.get_shape())
    
    #Second CNN layer
    W_conv2 = weight_variable([3, 3, 64, 128], "W_conv2")
    b_conv2 = bias_variable([128], "b_conv2")

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    
    print(h_conv2.get_shape())
    
    #Third CNN Layer
    W_conv3 = weight_variable([3, 3, 128, 128], "W_conv3")
    b_conv3 = bias_variable([128], "b_conv3")

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, strides2) + b_conv3)
    
    print(h_conv3.get_shape())
    
    #Fourth CNN Layer
    W_conv4 = weight_variable([3, 3, 128, 256], "W_conv4")
    b_conv4 = bias_variable([256], "b_conv4")

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4 ) + b_conv4)
    
    print(h_conv4.get_shape())
    
    #Fifth CNN Layer
    W_conv5 = weight_variable([3, 3, 256, 256], "W_conv5")
    b_conv5 = bias_variable([256], "b_conv5")

    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, strides2) + b_conv5)
    
    print(h_conv5.get_shape())
    
    #Sixth CNN Layer
    W_conv6 = weight_variable([3, 3, 256, 512], "W_conv6")
    b_conv6 = bias_variable([512], "b_conv6")

    h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)
    
    print(h_conv6.get_shape())
    
    #Mid Level features
    W_midlvl1 = weight_variable([3, 3, 512, 512], "W_midlvl1")
    b_midlvl1 = bias_variable([512], "b_midlvl1")

    h_midlvl1 = tf.nn.relu(conv2d(h_conv6, W_midlvl1) + b_midlvl1)
    #h_pool_midlvl1 = max_pool_2x2(h_midlvl1)

    print(h_midlvl1.get_shape())
    
    W_midlvl2 = weight_variable([3, 3, 512, 256], "W_midlvl2")
    b_midlvl2 = bias_variable([256], "b_midlvl2")

    h_midlvl2 = tf.nn.relu(conv2d(h_midlvl1, W_midlvl2) + b_midlvl2)

    print(h_midlvl2.get_shape())

    #Colorization Network
    
    #Transposed CNN Layer 1
    W_transpose_conv1 = weight_variable([3, 3, 128, 256], "W_transpose_conv1")
    b_transpose_conv1 = bias_variable([128], "b_transpose_conv1")

    h_conv_t_4 = conv2d_transpose_strided(h_midlvl2, W_transpose_conv1) + b_transpose_conv1
    
    print(h_conv_t_4.get_shape())
    
    # CNN conv
    
    W_colorizer_CNN1 = weight_variable([1, 1, 128 , 128], "W_colorizer_CNN1")
    b_colorizer_CNN1 = bias_variable([128], "b_colorizer_CNN1")

    h_colorizer_conv1 = tf.nn.relu6(conv2d(h_conv_t_4, W_colorizer_CNN1) + b_colorizer_CNN1)
    
    print(h_colorizer_conv1.get_shape())

    #Transposed CNN Layer 2
    W_transpose_conv2 = weight_variable([3, 3, 64, 128], "W_transpose_conv2")
    b_transpose_conv2 = bias_variable([64], "b_transpose_conv2")

    h_conv_t_5 = conv2d_transpose_strided(h_colorizer_conv1, W_transpose_conv2) + b_transpose_conv2
    
    print(h_conv_t_5.get_shape())
    
    
    # CNN conv
    
    W_colorizer_CNN2 = weight_variable([1, 1, 64 , 64], "W_colorizer_CNN2")
    b_colorizer_CNN2 = bias_variable([64], "b_colorizer_CNN2")

    h_colorizer_conv2 = tf.nn.relu6(conv2d(h_conv_t_5, W_colorizer_CNN2) + b_colorizer_CNN2)
    
    print(h_colorizer_conv2.get_shape())
    
      
    #Transposed CNN Layer 3
    W_transpose_conv3 = weight_variable([3, 3, 32, 64], "W_transpose_conv3")
    b_transpose_conv3 = bias_variable([32], "b_transpose_conv3")

    h_conv_t_6 = tf.add(conv2d_transpose_strided(h_colorizer_conv2, W_transpose_conv3), b_transpose_conv3)
    
    
    # CNN conv
    W_colorizer_CNN3 = weight_variable([1, 1, 32, 3], "W_colorizer_CNN3")
    b_colorizer_CNN3 = bias_variable([3], "b_colorizer_CNN3")

    h_colorizer_conv3 = tf.nn.relu6(conv2d(h_conv_t_6, W_colorizer_CNN3) + b_colorizer_CNN3)
    
    print(h_colorizer_conv3.get_shape())
    
    
    W_last = weight_variable([1, 1, 3, 3], "W_last")
    b_last = bias_variable([3], "b_last")
    last_cnn = tf.nn.sigmoid(conv2d(h_colorizer_conv3, W_last) + b_last, name="color_image")
    
    
    print(last_cnn.get_shape())

    #to_save = [W_conv1, b_conv1, W_conv2, b_conv2, W_conv3, b_conv3, W_transpose_conv1, b_transpose_conv1, W_transpose_conv2, b_transpose_conv2, W_transpose_conv3, b_transpose_conv3]    
    
    _lambda = 0.005;
  
    
    l1_regularizer = tf.contrib.layers.l1_regularizer(scale=_lambda, scope=None)
   
    weights = [W_conv1, W_conv2, W_conv3, W_conv4, W_conv5, W_conv6, W_midlvl1, W_midlvl2,
                    W_transpose_conv1, W_colorizer_CNN1, W_transpose_conv2, W_colorizer_CNN2, W_transpose_conv3,
                        W_colorizer_CNN3, W_last]
    
    regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, weights)
    
    #Loss computation
    gen_loss_mse = tf.reduce_mean(2 * tf.nn.l2_loss(last_cnn - y)) + regularization_penalty
    optimizer = tf.train.AdamOptimizer().minimize(gen_loss_mse)
    
    #pred image
    tf.add_to_collection('optimizer', optimizer)  
    tf.add_to_collection('loss', gen_loss_mse) 
    tf.add_to_collection('color_image', last_cnn)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    #Training
    for itr in range(hm_epochs):
        #image, color_images = get_next_batch(train_features, train_labels)
        image, color_images = get_next_batch_from_disk_RGB_Nocrop(images_list=images_list, batch_size=batch_size)
        #print(image.shape)
        feed_dict = {x: image, y: color_images}
        sess.run(optimizer, feed_dict=feed_dict)
        
        if itr % 100 == 0:
            mse = sess.run(gen_loss_mse, feed_dict=feed_dict)
            sess.run(optimizer, feed_dict=feed_dict)
            print("Step: %d, MSE: %g" % (itr, mse))
     
    saver.save(sess, model_path)
    
    sess.close()
'''